In [1]:
pip install transformers peft datasets accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
import torch


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `code_conv` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `code_con

In [4]:
# Load the base LLaMA model
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [32]:
from datasets import load_dataset

dataset = load_dataset("codeparrot/xlcost-text-to-code", "Python-program-level")

# Inspect dataset
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'code'],
        num_rows: 9263
    })
    test: Dataset({
        features: ['text', 'code'],
        num_rows: 887
    })
    validation: Dataset({
        features: ['text', 'code'],
        num_rows: 472
    })
})


In [50]:
dataset['train'][0]

{'text': 'Maximum Prefix Sum possible by merging two given arrays | Python3 implementation of the above approach ; Stores the maximum prefix sum of the array A [ ] ; Traverse the array A [ ] ; Stores the maximum prefix sum of the array B [ ] ; Traverse the array B [ ] ; Driver code',
 'code': 'def maxPresum ( a , b ) : NEW_LINE INDENT X = max ( a [ 0 ] , 0 ) NEW_LINE for i in range ( 1 , len ( a ) ) : NEW_LINE INDENT a [ i ] += a [ i - 1 ] NEW_LINE X = max ( X , a [ i ] ) NEW_LINE DEDENT Y = max ( b [ 0 ] , 0 ) NEW_LINE for i in range ( 1 , len ( b ) ) : NEW_LINE INDENT b [ i ] += b [ i - 1 ] NEW_LINE Y = max ( Y , b [ i ] ) NEW_LINE DEDENT return X + Y NEW_LINE DEDENT A = [ 2 , - 1 , 4 , - 5 ] NEW_LINE B = [ 4 , - 3 , 12 , 4 , - 3 ] NEW_LINE print ( maxPresum ( A , B ) ) NEW_LINE'}

In [33]:
# Set the pad_token to eos_token if it's not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


In [34]:
def preprocess_function(examples):
    # Extract the text and code fields from the batch
    inputs = examples['text']  # This will be a list of input strings
    targets = examples['code']  # This will be a list of target strings

    # Tokenize the inputs
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize the targets and assign them as labels
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length").input_ids
    model_inputs["labels"] = labels

    return model_inputs


In [35]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/9263 [00:00<?, ? examples/s]

Map:   0%|          | 0/887 [00:00<?, ? examples/s]

Map:   0%|          | 0/472 [00:00<?, ? examples/s]

In [36]:
# Split the tokenized dataset if not already split
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]


In [37]:
# Check the size of the datasets
print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(eval_dataset)}")


Train dataset size: 9263
Eval dataset size: 472


In [38]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM

# Configure PEFT with LoRA
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,  # Disable inference mode during training
    r=16,                  # Rank for LoRA layers
    lora_alpha=32,         # Scaling factor
    lora_dropout=0.1       # Dropout rate for LoRA
)

# Load the LLaMA model
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B",
    load_in_8bit=True,  # Use 8-bit quantization for efficiency
    device_map="auto"
)

# Apply LoRA to the model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 1,703,936 || all params: 1,237,518,336 || trainable%: 0.1377


In [39]:
from transformers import TrainingArguments

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./llama-finetuned",  # Output directory to save the model
    per_device_train_batch_size=4,   # Batch size per device (GPU)
    gradient_accumulation_steps=8,   # Number of steps to accumulate gradients
    num_train_epochs=3,              # Number of training epochs
    learning_rate=2e-4,              # Learning rate
    fp16=True,                       # Mixed precision training
    logging_dir="./logs",            # Directory to store logs
    logging_steps=50,                # Log every 10 steps
    save_steps=500,                  # Save model every 500 steps
    evaluation_strategy="steps",     # Evaluate after each set of steps
    save_total_limit=2,              # Keep only the last two saved checkpoints
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [40]:
from transformers import Trainer

# Initialize the Trainer with the model, arguments, and datasets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)


<ipython-input-40-8088d0ae87d7>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [41]:
# Train the model
trainer.train()


Step,Training Loss,Validation Loss
50,4.155500,2.661066
100,2.559300,2.485895
150,2.500400,2.410522
200,2.458100,2.389422
250,2.350900,2.399568
300,2.426100,2.404459
350,2.424300,2.369062
400,2.424600,2.336894
450,2.307300,2.345793
500,2.326100,2.327770


TrainOutput(global_step=867, training_loss=2.480753743524782, metrics={'train_runtime': 2162.9509, 'train_samples_per_second': 12.848, 'train_steps_per_second': 0.401, 'total_flos': 8.308005897712435e+16, 'train_loss': 2.480753743524782, 'epoch': 2.994818652849741})

In [42]:
# Save the fine-tuned model and tokenizer
model.save_pretrained("./llama-finetuned")
tokenizer.save_pretrained("./llama-finetuned")


('./llama-finetuned/tokenizer_config.json',
 './llama-finetuned/special_tokens_map.json',
 './llama-finetuned/tokenizer.json')

In [43]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 2.311253786087036, 'eval_runtime': 12.5288, 'eval_samples_per_second': 37.673, 'eval_steps_per_second': 4.709, 'epoch': 2.994818652849741}


In [49]:
from transformers import pipeline

# Load the fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained("./llama-finetuned")
tokenizer = AutoTokenizer.from_pretrained("./llama-finetuned")

# Inference pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# Test input (code generation prompt)
text = "Maximum Prefix Sum possible by merging two given arrays | Python3 implementation of the above approach ; "
output = generator(text, max_length=100, num_return_sequences=1)

print(output[0]['generated_text'])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Maximum Prefix Sum possible by merging two given arrays | Python3 implementation of the above approach ; _LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE_LINE


In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
save_path = '/content/drive/MyDrive/llama-finetuned'

In [47]:
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:418: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


('/content/drive/MyDrive/llama-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/llama-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/llama-finetuned/tokenizer.json')